<a href="https://colab.research.google.com/github/Musa333333/business-assistant/blob/main/accurate_response.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:

from google.colab import userdata
from huggingface_hub import login

login(token=userdata.get('HF-TOKEN'))  # Use your HF Token

In [ ]:

from unsloth import FastLanguageModel
from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    token=userdata.get('HF-TOKEN'),
)

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,
    loftq_config=None,


)

Unsloth 2025.2.15 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a business expert with advanced knowledge in Accounting, Banks and Banking, Business, Business Communication, Career management, E-Business, Economics, Entrepreneurship, Finance, Financial Planning, Industries and Professions, Investments, Management and Leadership, Marketing and Sales, Real Estate, Stock Trading, Tax, Small Business, and Entrepreneurship.
Please answer the following medical question.

### Question:
{}

### Response:
<think>
{}
</think>
{}"""

In [ ]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    inputs = examples["prompt"]
    cots = examples["thinking"]
    outputs = examples["completion"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }


In [ ]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/fine_tuning_data5.jsonl", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/894 [00:00<?, ? examples/s]

In [ ]:
dataset["text"][0]

'Below is an instruction that describes a task, paired with an input that provides further context.\nWrite a response that appropriately completes the request.\nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a business expert with advanced knowledge in Accounting, Banks and Banking, Business, Business Communication, Career management, E-Business, Economics, Entrepreneurship, Finance, Financial Planning, Industries and Professions, Investments, Management and Leadership, Marketing and Sales, Real Estate, Stock Trading, Tax, Small Business, and Entrepreneurship.\nPlease answer the following medical question.\n\n### Question:\nlack of visibility into financial performance across different departments\n\n### Response:\n<think>\naccounting\n</think>\nimplement a financial performance dashboard with department-specific data and analytics<｜end▁of▁sentence｜>'

In [ ]:

def is_business_related(question):
    # Define expanded keywords for various business-related categories
    business_keywords = [
        "accounting", "bookkeeping", "audit", "financial statement", "ledgers", "tax filing", "balance sheet", "revenue",
        "expenses", "profit", "liabilities", "assets", "accounts receivable", "accounts payable",  # Accounting
        "bank", "banking", "loans", "credit", "savings", "interest rates", "mortgage", "account management", "transactions",
        "bank fees", "digital banking", "financial institution", "deposit", "withdrawal", "business", "sales", "strategy", "growth", "payment", "improvement", "revenue", "startup", "operations",
        "entrepreneurship", "management", "finance", "marketing", "sales", "investment", "market", "payment gateway",  # Banks and Banking
        "business", "strategy", "growth", "startup", "operations", "team management", "business model", "venture",
        "innovation", "profit", "scale", "market share", "industry", "competition",  # Business
        "communication", "email", "meeting", "negotiation", "public relations", "corporate communication", "branding",
        "presentations", "messaging",  # Business Communication
        "career", "job search", "resume", "skills", "job interview", "career development", "promotion", "networking",
        "professional growth", "career path",  # Career Management
        "e-commerce", "online store", "digital marketing", "online business", "website", "payment gateway", "online transaction",
        "digital sales", "product listings", "dropshipping", "affiliate marketing", "e-business model",  # E-Business
        "economics", "market", "supply and demand", "inflation", "GDP", "trade", "fiscal policy", "monetary policy",
        "recession", "growth", "economic theory", "economic trends", "market equilibrium",  # Economics
        "entrepreneur", "startup", "business plan", "funding", "innovation", "pitching", "venture capital", "small business",
        "founder", "leadership", "scalability", "risk-taking", "idea generation",  # Entrepreneurship
        "finance", "investments", "capital", "assets", "liabilities", "equity", "risk", "portfolio", "stock market",
        "bonds", "savings", "returns", "financial analysis", "wealth management",  # Finance
        "financial planning", "budgeting", "savings plan", "retirement", "financial goals", "investment strategies",
        "debt management", "financial advisor", "wealth planning", "estate planning", "tax planning",  # Financial Planning
        "industry", "profession", "career", "job market", "manufacturing", "technology", "healthcare", "retail", "education",
        "service sector", "labor market", "sector", "business professional",  # Industries and Professions
        "investment", "stocks", "bonds", "mutual funds", "ETF", "portfolio management", "risk assessment", "diversification",
        "asset allocation", "real estate investment", "venture capital", "angel investors",  # Investments
        "management", "leadership", "team management", "project management", "decision-making", "strategic planning",
        "organizational structure", "performance", "coaching", "mentoring",  # Management and Leadership
        "marketing", "sales", "advertising", "branding", "content marketing", "SEO", "SEM", "social media", "customer acquisition",
        "lead generation", "market research", "sales funnel", "digital marketing",  # Marketing and Sales
        "real estate", "property", "investment property", "rental", "leasing", "market trends", "commercial real estate",
        "residential property", "real estate agent", "real estate market", "home buying", "property management",  # Real Estate
        "stock trading", "shares", "portfolio", "stock market", "day trading", "brokerage", "stock analysis", "trading strategy",
        "market trends", "investment", "financial markets", "equity", "risk management",  # Stock Trading
        "tax", "taxation", "income tax", "corporate tax", "tax filing", "tax returns", "deductions", "tax planning", "tax strategy",
        "VAT", "tax bracket", "corporate tax rate", "tax laws", "compliance",  # Tax
        "small business", "entrepreneur", "startup", "business plan", "market research", "funding", "business development",
        "small business strategy", "scaling", "operations", "cash flow", "small business management", "business ownership"  # Small Business and Entrepreneurship
    ]
    return any(keyword in question.lower() for keyword in business_keywords)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        num_train_epochs = 2, warmup_ratio = 0.1,
        warmup_steps=5,
        max_steps=200,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to = "none", # Use this for WandB etc
    ),
)



Map:   0%|          | 0/894 [00:00<?, ? examples/s]

Converting train dataset to ChatML (num_proc=2):   0%|          | 0/894 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/894 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/894 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/894 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 894 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
10,2.814500
20,2.089900
30,1.994000
40,1.802900
50,1.809100
60,1.491600


In [ ]:
prompt_style ="""Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a business expert with advanced knowledge in Accounting, Banks and Banking, Business, Business Communication, Career management, E-Business, Economics, Entrepreneurship, Finance, Financial Planning, Industries and Professions, Investments, Management and Leadership, Marketing and Sales, Real Estate, Stock Trading, Tax, Small Business, and Entrepreneurship.
Please answer the following medical question.


### Question:
{}

### Response:
<think>{}"""

In [ ]:
question = "Tell me a joke about cats marketing in business."

if is_business_related(question):
    FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
    inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=1200,
        use_cache=True,
    )
    response = tokenizer.batch_decode(outputs)
    print(response[0].split("### Response:")[1])
else:
    print("The question is not business-related.")


<think>
Okay, so I need to create a joke about cats marketing in business. Hmm, let's start by brainstorming some cat-related business scenarios. Maybe something about a cat opening a marketing firm or getting into business with other animals. I want the joke to be relatable and funny, but also tie into marketing concepts. Perhaps the cat uses clever marketing strategies that lead to funny business situations. I should make sure the joke has a clear punchline and ties back to business or marketing in a humorous way. Let me think of a cat named Max who starts a marketing firm. He uses clever strategies, maybe something unexpected, and ends up in a funny situation. The punchline should be something memorable. I need to make sure the joke flows well and is easy to understand. Maybe involve other animals as clients or competitors. Let me structure the joke with a setup and a punchline that ties into a business or marketing concept. I think that's a good approach. Time to put it all togeth